# ⚡ MOSFET 102

In this notebook, we will cover more advanced MOSFET simulation and complete a task that all analog IC designers begin with when they encounter a new PDK: pipecleaning simulations. These are done to provide the parameters for the transistor models that analog designers use to intuitively navigate their design space:

🎥 Video Insight:

<iframe width="560" height="315" src="https://www.youtube.com/embed/HYjzjqk54hk?si=0JArH0DdcgYxyr8g" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>

We'll use this as an opportunity to learn about both the models, as well as demonstrating how we extract the necessary parameters to use them in practice.

### Basic Transistor Operation: A Quick Recap

📝 To ensure we're all on the same page, here's a concise summary of the video's insights:

| | Subthreshold | Strong Inversion | Mobility Degradation | 
| --- | --- | --- | --- |
| Region of validity | $V_{eff} \leq 0$ | $\dfrac{2nkT}{q} < V_{eff} < \dfrac{1}{2\theta}$ | $V_{eff} > \dfrac{1}{2\theta}$ |
| Drain current, $I_D$ | $I_{D0}\left(\dfrac{W}{L}\exp(qV_{eff}/nkT)\right)$ | $\dfrac12 \mu_n C_{ox}\dfrac{W}{L}V_{eff}^2$ | $\dfrac{\mu_n C_{ox} (W/L)V_{eff}^2}{2[1+(\theta V_{eff}^m)]^{1/m}}$ |
| Small-signal transconductance, $g_m$ | $\dfrac{qI_D}{nkT}$ | $\dfrac{2I_D}{V_{eff}} = \mu_n C_{ox} \dfrac{W}{L} V_{eff}$ | $\dfrac12 \mu_n C_{ox} \dfrac{W}{L\theta}$

The variables in question are described in the table below:

| Variable | Explanation |
| --- | --- |
| $ V_{eff} $ | Effective voltage across the transistor |
| $ V_{gs} $ | Gate-Source Voltage |
| $ V_{th} $ | Threshold Voltage |
| $ n $ | Subthreshold slope factor, typically around 1.5 for MOSFETs |
| $ kT $ | Thermal voltage, where $ k $ is the Boltzmann constant and $ T $ is the absolute temperature |
| $ q $ | Charge of an electron |
| $ \theta $ | Mobility degradation coefficient |
| $ I_{D0} $ | Drain current at the threshold voltage |
| $ W $ | Width of the transistor |
| $ L $ | Length of the transistor |
| $ I_D $ | Drain current |
| $ \mu_n $ | Electron mobility |
| $ C_{ox} $ | Oxide capacitance per unit area |
| $ g_m $ | Small-signal transconductance |
| $ m $ | Exponent in the mobility degradation equation, typically around 1.5 to 2.5 |
 
$V_{eff}$ is usually defined as $V_{gs}-V_{th}$.

Now, with our foundation set, let's dive into the heart of the matter!

### Simulating Subthreshold Operation

We'll start by investigating the subthreshold operation of a MOSFET, notice in the plot below that you get the exponential behavior expected and that the slope of the line between $V_{gs}=0.3$ and $V_{gs}=0.8$ on linear-log axes is $q/nkT$.

##### 🧠 Exercises

1. Play with the device provided or substitute your own, is the subthreshold behavior effected how you'd imagine?
2. Compute the critical subthreshold parameters $q/nkT$ and $I_{D0}$
3. Where is subthreshold current leakage an important consideration in design - how can we mitigate it?

In [ ]:
# Import the hdl21 library
import hdl21 as h
from hdl21.prefix import u
import vlsirtools.spice as vsp

# Import the SKY130 PDK
from sitepdks import *
import gf180_hdl21 as g
from gf180_hdl21 import primitives as p

# Import matplotlib for plotting
import matplotlib.pyplot as plt

# The minimum gate length of SKY130
min_gate_length = 0.18 * u

# Three devices to characterize our MOSFETs
device1 = p.NFET_3p3V(w=15*min_gate_length, l=min_gate_length, nf=15)
device2 = p.NFET_3p3V(w=15*min_gate_length, l=2*min_gate_length, nf=15)
device3 = p.NFET_3p3V(w=15*min_gate_length, l=10*min_gate_length, nf=15)

# Stick 'em in a list
devices = [device1, device2, device3]

def get_sim(device : h.Module,vds=0.1) -> h.sim.Sim:

    @h.sim.sim
    class DeviceSim:

        @h.module
        class Tb:

            # Set up our ports
            VSS = h.Port()
            VDD = h.Signal()
            
            # Define our voltages
            vdd = h.DcVoltageSource(dc=vds)(p=VDD, n=VSS)
            sweep = h.PulseVoltageSource(delay=0,
                                v1=0,
                                v2=1.8,
                                period=1,
                                rise=1,
                                fall=1,
                                width=1)(n=VSS)
            # Instantiate our device
            DUT = device(d=VDD, g=sweep.p, s=VSS, b=VSS)

        tran = h.sim.Tran(tstop=1,tstep=0.001)
        inc = g.install.include(h.pdk.Corner.TYP)

    return DeviceSim

opts = vsp.SimOptions(
            simulator=vsp.SupportedSimulators.NGSPICE,
            fmt=vsp.ResultFormat.SIM_DATA,
            rundir="./scratch",
        )

# Iterate over the devices and simulate
vds=0.1
sim = get_sim(device=device2,vds=vds)
rvs = sim.run(opts=opts)

# Extract data
x = rvs.an[0].data['v(xtop.sweep_p)']
y = -rvs.an[0].data['i(v.xtop.vvdd)']

plt.plot(x,y)
plt.title(f"Subthreshold MOSFET I-V Curve at "+'$V_{ds}=$'+f"{vds} V")
plt.xlabel("$V_{gs}$ (V)")
plt.ylabel("$I_D$ (A)")
plt.yscale("log")

### Simulating Subthreshold Transconductance

We're told from the active mode $g_m$ approximation that $g_m = 2I_D/V_{eff}$ - so as $V_{eff}$ approaches $0$, we'd expect $g_m$ to explode to infinity - as shown in the lecture, this isn't what happens, $g_m$ is instead limited by subthreshold operation to a maximum linear relationship with $I_D$ with a slope of $q/nkT$.

The curve in the lecture looks at the $g_m/I_D$ ratio against $V_{gs}$, and we can compute these values ourselves for the above MOSFET using the data from the simulation we just performed.

##### 🧠 Exercises

1. Identify $q/nkT$ and $V_{th}$ from the graph below
2. How does modifying parameters effect this curve, what about temperature?
3. What does this tell us about how we can maximize $g_m$ in our designs, how can realize this in reality?

In [ ]:
import numpy as np

gm = np.gradient(y,x)[10:]
gm_id = gm/y[10:]

plt.xlabel("$V_{gs}$ ($V$)")
plt.ylabel("$g_m/I_D$ ($V^{-1}$)")
plt.plot(x[10:],gm_id)

### Simulating Mobility Degradation

Mobility degradation occurs as $V_{eff}$ gets so high that new scattering and saturation phenomena occur that degrade the mobility of our charge carrier, this is reflected in the more exotic quotient of the mobility degradation equation.

##### 🧠 Exercises

1. Tweak the simulation, how do you expect it to react, does it do what you expect?
2. Find a way to manipulate the simulation data to extract $\theta$ (hint: think $g_m$)
3. Assuming you've computed $\theta$, compute $m$.

In [ ]:
# Don't touch any of this!
def get_sim(device : h.Module,vgs=1.8) -> h.sim.Sim:

    @h.sim.sim
    class DeviceSim:

        @h.module
        class Tb:

            # Set up our ports
            VSS = h.Port()
            VDD = h.Signal()
            
            # Define our voltages
            vdd = h.DcVoltageSource(dc=vgs)(p=VDD, n=VSS)
            sweep = h.PulseVoltageSource(delay=0,
                                         v1=2.5,
                                         v2=5,
                                         period=1,
                                         rise=1,
                                         fall=1,
                                         width=1)(n=VSS)
            # Instantiate our device
            DUT = device(d=sweep.p, g=VDD, s=VSS, b=VSS)

        tran = h.sim.Tran(tstop=1,tstep=0.001)
        inc1 = g.install.include_design()
        inc2 = g.install.include_mos(h.pdk.CmosCorner.TT)

    return DeviceSim

opts = vsp.SimOptions(
            simulator=vsp.SupportedSimulators.NGSPICE,
            fmt=vsp.ResultFormat.SIM_DATA,
            rundir="./scratch",
        )

# Modify vgs, W/L and simulate!
vgs=1.8
w, l = 0.6*u, 0.2*u
sim = get_sim(device=p.NMOS_3p3V(w=w, l=l),vgs=vgs)
rvs = sim.run(opts=opts)

results = rvs[vsp.sim_data.AnalysisType.TRAN]

# Plot the results
x = results.data['v(xtop.sweep_p)']
y = -results.data['i(v.xtop.vsweep)']*1000

plt.xlabel('Vds (V)')
plt.ylabel('Ids (mA)')
plt.title(f'Mobility Degradation Curve for {w}/{l} NMOS at Vgs={vgs}V')

plt.plot(x,y)

### Computing $V_{th}$ and $g_m$: A Guided Exploration

Our journey begins with two pivotal parameters: $V_{th}$ and $g_m$. Extracting these values is a systematic process:

- **Device Selection**: Choose a range of devices that pique your interest.
- **Model-Based Testing**: Leverage our models to craft tests and pinpoint the desired values.

For instance, to unearth $V_{th}$, we'll keep $V_{ds}$ elevated and sweep through $V_{gs}$ across our chosen devices. This approach yields a $V_{gs}/I_D$ curve, enabling us to compute both $V_{th}$ and $g_m$.

##### 🧠 Exercises
1. **Parameter Exploration**: Identify the point corresponding to $V_{th}$
2. **Parameter Exploration**: Determine $g_m$ from the plots

In [ ]:
def get_sim(device : h.Module) -> h.sim.Sim:

    @h.sim.sim
    class DeviceSim:

        @h.module
        class Tb:

            # Set up our ports
            VSS = h.Port()
            VDD = h.Signal()
            
            # Define our voltages
            vdd = h.DcVoltageSource(dc=1.8)(p=VDD, n=VSS)
            sweep = h.PulseVoltageSource(delay=0,
                                            v1=0,
                                            v2=1.8,
                                            period=1,
                                            rise=1,
                                            fall=1,
                                            width=1)(n=VSS)
            # Instantiate our device
            DUT = device(d=VDD, g=sweep.p, s=VSS, b=VSS)

        tran = h.sim.Tran(tstop=1,tstep=0.001)
        inc1 = g.install.include_design()
        inc2 = g.install.include_mos(h.pdk.Corner.TYP)

    return DeviceSim

opts = vsp.SimOptions(
            simulator=vsp.SupportedSimulators.NGSPICE,
            fmt=vsp.ResultFormat.SIM_DATA,
            rundir="./scratch",
        )

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Iterate over the devices and simulate
sim = get_sim(device=device2)
rvs = sim.run(opts=opts)

# Extract data
x = rvs.an[0].data['v(xtop.sweep_p)']
y = -rvs.an[0].data['i(v.xtop.vvdd)']*1000

# Compute the derivative
dy_dx = np.gradient(y, x)

# Plot the original data
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.plot(x, y, label="$W/L=15/2, V_{ds} = 1.8V$")
plt.xlabel('$V_{gs}$ (V)')
plt.ylabel('$I_D$ (mA)')
plt.title('15/2 MOSFET Characteristic')
plt.legend()

# Plot the derivative
plt.subplot(1, 3, 2)
plt.plot(x, dy_dx, label="$W/L=15/2, V_{ds} = 1.8V$")
plt.xlabel('$V_{gs}$ (V)')
plt.ylabel('$g_m$ (mA/V)')
plt.title('15/2 MOSFET Transconductance')
plt.legend()

# Plot the second derivative
plt.subplot(1,3,3)
plt.plot(x, np.gradient(dy_dx, x), label="$W/L=15/2, V_{ds} = 1.8V$")
plt.xlabel('$V_{gs}$ (V)')
plt.ylabel("$g_m'$ (mA/V$^2$)")
plt.title('15/2 MOSFET Transconductance Derivative')
plt.legend() 


plt.tight_layout()
plt.show()


In [ ]:
# Iterate over the devices and simulate
sim = get_sim(device=device3)
rvs = sim.run(opts=opts)

# Extract data
x = rvs.an[0].data['v(xtop.sweep_p)']
y = -rvs.an[0].data['i(v.xtop.vvdd)']*1000

# Compute the derivative
dy_dx = np.gradient(y, x)

# Plot the original data
plt.figure(figsize=(10, 5))
plt.subplot(1, 3, 1)
plt.plot(x, y, label="$W/L=15/10, V_{ds} = 1.8V$")
plt.xlabel('$V_{gs}$ (V)')
plt.ylabel('$I_D$ (mA)')
plt.title('15/10 MOSFET Characteristic')
plt.legend()

# Plot the derivative
plt.subplot(1, 3, 2)
plt.plot(x, dy_dx, label="$W/L=15/10, V_{ds} = 1.8V$")
plt.xlabel('$V_{gs}$ (V)')
plt.ylabel('$g_m$ (mA/V)')
plt.title('15/10 MOSFET Transconductance')
plt.legend()

# Plot the second derivative
plt.subplot(1,3,3)
plt.plot(x, np.gradient(dy_dx, x), label="$W/L=15/10, V_{ds} = 1.8V$")
plt.xlabel('$V_{gs}$ (V)')
plt.ylabel("$g_m'$ (mA/V$^2$)")
plt.title('15/10 MOSFET Transconductance Derivative')
plt.legend() 



plt.tight_layout()
plt.show()

### Finding $r_{ds}$ and $\lambda$

Now, this is where things get a tad more challenging. But with our previous success, we're more than equipped! By running a simulation with a high $V_{gs}$ and sweeping $V_{ds}$, the resulting $I_{D}/V_{ds}$ curve and its derivative become our guiding stars to compute these values.

##### 🧠 Exercises:

1. Determine $\lambda$ from the plots below.
2. Determine $r_{ds}$ from the plots below
3. Explore the impact of varying $V_{gs}$ on $r_{ds}$. How does this variation influence the overall behavior of the MOSFET, you may want to look at the small signal model

In [ ]:
def get_sim(device : h.Module) -> h.sim.Sim:

    @h.sim.sim
    class DeviceSim:

        @h.module
        class Tb:

            # Set up our ports
            VSS = h.Port()
            VDD = h.Signal()
            
            # Define our voltages
            vdd = h.DcVoltageSource(dc=1.8)(p=VDD, n=VSS)
            sweep = h.PulseVoltageSource(delay=0,
                                         v1=0,
                                         v2=1.8,
                                         period=1,
                                         rise=1,
                                         fall=1,
                                         width=1)(n=VSS)
            # Instantiate our device
            DUT = device(d=sweep.p, g=VDD, s=VSS, b=VSS)

        tran = h.sim.Tran(tstop=1,tstep=0.001)
        inc = s.install.include(h.pdk.Corner.TYP)

    return DeviceSim

opts = vsp.SimOptions(
            simulator=vsp.SupportedSimulators.NGSPICE,
            fmt=vsp.ResultFormat.SIM_DATA,
            rundir="./scratch",
        )

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Iterate over the devices and simulate
sim = get_sim(device=device2)
rvs = sim.run(opts=opts)

# Extract data
x = rvs.an[0].data['v(xtop.sweep_p)']
y = -rvs.an[0].data['i(v.xtop.vsweep)']*1000

# Compute the derivative
dy_dx = np.gradient(y, x)

# Plot the original data
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.plot(x, y, label="$W/L=15/2, V_{gs} = 1.8V$")
plt.xlabel('$V_{ds}$ (V)')
plt.ylabel('$I_D$ (mA)')
plt.title('15/2 MOSFET Characteristic')
plt.legend()

# Plot the derivative
plt.subplot(1, 3, 2)
plt.plot(x, dy_dx, label="$W/L=15/2, V_{gs} = 1.8V$")
plt.xlabel('$V_{ds}$ (V)')
plt.ylabel('$g_{ds}$ (mA/V)')
plt.title('15/2 MOSFET Transconductance')
plt.legend()

# Plot the second derivative
plt.subplot(1,3,3)
plt.plot(x[1:], (dy_dx/y)[1:], label="$W/L=15/2, V_{gs} = 1.8V$")
plt.xlabel('$V_{ds}$ (V)')
plt.ylabel("$\dfrac{g_{ds}}{I_{D}}$ (V$^{-1}$)")
plt.title('15/2 MOSFET Transconductance Derivative')
plt.legend()

plt.ylim(0, 1)

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Iterate over the devices and simulate
sim = get_sim(device=device3)
rvs = sim.run(opts=opts)

# Extract data
x = rvs.an[0].data['v(xtop.sweep_p)']
y = -rvs.an[0].data['i(v.xtop.vsweep)']*1000

# Compute the derivative
dy_dx = np.gradient(y, x)

# Plot the original data
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.plot(x, y, label="$W/L=15/10, V_{gs} = 1.8V$")
plt.xlabel('$V_{ds}$ (V)')
plt.ylabel('$I_D$ (mA)')
plt.title('15/10 MOSFET Characteristic')
plt.legend()

# Plot the derivative
plt.subplot(1, 3, 2)
plt.plot(x, dy_dx, label="$W/L=15/10, V_{gs} = 1.8V$")
plt.xlabel('$V_{ds}$ (V)')
plt.ylabel('$g_{ds}$ (mA/V)')
plt.title('15/10 MOSFET Transconductance')
plt.legend()

# Plot the second derivative
plt.subplot(1,3,3)
plt.plot(x[1:], (dy_dx/y)[1:], label="$W/L=15/10, V_{gs} = 1.8V$")
plt.xlabel('$V_{ds}$ (V)')
plt.ylabel("$\dfrac{g_{ds}}{I_{D}}$ (V$^{-1}$)")
plt.title('15/10 MOSFET Transconductance Derivative')
plt.legend()

plt.ylim(0, 1)

plt.tight_layout()
plt.show()